# 2D DC Resistivity Inversion

In [1]:
from dc_utils.DCResistivity import DCRInversionApp
import pandas as pd
import numpy as np
from simpeg.electromagnetics.static.utils import geometric_factor
dc_app = DCRInversionApp()

# Purpose

We load observed DC data, and invert to recover a resistivity model. By altering inversion parameters and exploring correspoding inversion results, we investigate important aspects of the 2D DC resistivity inversion. 

## Outline
This notebook includes four steps:
- Step1: Load observed data
- Step2: Plot observed data
- Step3: Set mesh
- Step4: Set uncertainty
- Step5: Run inversion
- Step6: Explore inversion results
- Step7: Run inversion to compute DOI index
- Step8: Plot DOI index

## Step 1: Load observed data

- `A`: A electrode locations in a shape (*, 2) (x and z positions for each electrode)
- `B`: B electrode locations in a shape (*, 2) (x and z positions for each electrode)
- `M`: M electrode locations in a shape (*, 2) (x and z positions for each electrode)
- `N`: N electrode locations in a shape (*, 2) (x and z positions for each electrode)
- `rho_a`: The apparent resistivity in (ohm m)

In [8]:
data1 = pd.read_csv(
    'data/Wenner_1-2023-03-06-104643.txt', sep='\t',skiprows=483, nrows=991)

#data1.drop(6, inplace=True)

A = np.c_[data1['A(x)'],data1['A(z)']]
B = np.c_[data1['B(x)'],data1['B(z)']]
M = np.c_[data1['M(x)'],data1['M(z)']]
N = np.c_[data1['N(x)'],data1['N(z)']]
rho1 = np.asarray(data1['App.R(Ohmm)'])

includes = (rho1 <= 1000)

A = A[includes]
B = B[includes]
M = M[includes]
N = N[includes]

rho1 = rho1[includes]

In [9]:
data1

,N,Time,MeasID,Seq#,DPID,Channel,A(x),A(y),A(z),B(x),...,IP delay window(mV/V),IP#1(mV/V),IP#2(mV/V),Pint(V),Pext(V),Temp(C),Latitude,Longitude,Pos.Quality,Unnamed: 40
0,1,2023-03-06 08:57:48,528,2,2143,1,0.0,0.0,0.0,15.0,...,-88.517800,28.3938,24.2530,0.0,12.2,52.3,40.489500,-106.841742,3,NaN
1,2,2023-03-06 08:59:57,547,2,2379,1,5.0,0.0,0.0,20.0,...,19.855400,35.0801,29.0489,0.0,12.3,52.9,40.489473,-106.841730,3,NaN
2,3,2023-03-06 08:51:39,474,2,2163,1,0.0,0.0,0.0,30.0,...,374.758000,49.2240,40.3393,0.0,11.1,52.5,40.489470,-106.841703,3,NaN
3,4,2023-03-06 09:02:06,566,2,2189,1,10.0,0.0,0.0,25.0,...,-0.936714,37.7840,31.6821,0.0,12.2,53.1,40.489497,-106.841728,3,NaN
4,5,2023-03-06 08:52:40,483,2,2408,1,5.0,0.0,0.0,35.0,...,234.406000,52.5949,43.1273,0.0,12.3,52.5,40.489488,-106.841730,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,987,2023-03-06 10:24:09,982,2,4606,1,440.0,0.0,0.0,470.0,...,164.428000,72.7698,61.5547,0.0,11.9,47.7,40.488403,-106.840567,3,NaN
987,988,2023-03-06 10:26:09,1000,2,5036,1,450.0,0.0,0.0,465.0,...,92.558200,35.7481,30.2351,0.0,12.0,47.8,40.488403,-106.840532,3,NaN
988,989,2023-03-06 10:24:29,985,2,4998,1,445.0,0.0,0.0,475.0,...,172.368000,89.1559,75.9801,0.0,12.0,47.7,40.488417,-106.840562,3,NaN
989,990,2023-03-06 10:26:43,1005,2,4962,1,455.0,0.0,0.0,470.0,...,92.310600,38.2361,32.9163,0.0,12.0,47.8,40.488428,-106.840563,3,NaN


In [10]:
dc_app.load_obs(A, B, M, N, rho1)

>> survey type: dipole-dipole
   # of data: 990
>> suggested initial resistivity: 15 ohm-m


## Step 2: Plot observed data

Once loaded, we examine the data. From the *pseudo-section* plot, we can determine the background resistivity of the Earth and highlight any interesting features. To view the data, we can use the following parameters:

- `data type`: type of the data
- `plot type`: type of the data plot
- `aspect ratio`: aspect ratio for pseudo-section plot. Changes size of plot

**IMPORTANT:** The data loaded into this notebook are the measured voltages. We plot the data as apparent resistivities when we want an easier first interpretation. 

In [11]:
dc_app.interact_plot_obs_data()

interactive(children=(ToggleButtons(description='data type', options=('apparent_resistivity', 'volt'), value='…

## Step 3: Set a 2D mesh 

Designing an mesh is very important. If we use smaller cells in our mesh, we solve the problem more accurately. However, we cannot make the cells too small because it requires too much computer memory. Parameters for designing a mesh are:

- `dx`: horizontal cell size
- `dz`: vertical cell size
- `corezlength`: the depth that the core mesh extends
- `xpad`: thickness of padding region in the horizontal direction
- `zpad`: thickness of padding region in the vertical direction
- `mesh_type`: tensor or tree mesh

Some things to consider when designing a mesh:

- Using a 'Tree Mesh' will **always** reduce the number of cells.


- To choose a good *dx* value, think about the minimum electrode spacing. The value of *dx* should be 25%-40% as large as the minimum electrode spacing. So if minimum electrode spacing is 10 m, you should choose a *dx* between 2.5 m and 4 m.


- It is common to let *dz* be the same as *dx*. Sometimes we set *dz* to be half the size of *dx*.


- *corezlength* defines a depth limit for where you think most currents are. Below this depth, you are assuming the currents are very small. A good first guess would be to set *corezlength* to be equal to the maximum AB electrode spacing.


- The padding (*xpad* and *zpad*) needs to be big enough so that the currents are zero on the edges of your mesh. For a Wenner-Schlumberger survey, set the padding region to be 1-2 times larger than the maximum AB spacing. For a dipole-dipole survey, set the padding region to be 2-3 times larger than the maximum AB spacing

In [12]:
dc_app.interact_set_mesh()

dx is set to 1.25 m (samllest electrode spacing (5.0) / 4)
dz (0.625 m) is set to dx (1.25 m) / 2
>> suggested dx: 1.25 m
>> suggested dz: 0.625 m
>> suggested x padding: 2.6 m
>> suggested z padding: 2.6 m
>> suggested corezlength: 1.3 m


interactive(children=(FloatText(value=1.25, description='dx'), FloatText(value=0.625, description='dz'), Float…

## Step 4: Set uncertainty

The uncertainties are an estimate of the level of noise on your data. If your data have more noise (error bars are larger), you must assign larger uncertainties. The equation for the uncertainty you assign to each data value is:

$$ \text{uncertainty} = 0.01 \times \text{percentage}\times|d^{obs}| + \text{floor}$$

- **percentage (%):** percentage uncertainty
- **floor (V):** floor uncertainty. The raw data are volts, not apparent resistivities.

**The data are measured voltages (not apparent resistivities)**. So we have to apply uncertainties on the voltage values. Here are some things to consider when choosing uncertainties:

- For DC resistivity data, it is common to choose a percentage between 2%-10%. A higher percentage is chosen if your data are more noisy.
- Measurements with large electrode spacings give you information at larger depth. But since the signal is weaker, these data are more noisy. We must apply a floor uncertainty so we do not overfit these small voltage values.
- Higher uncertainties are usually chosen for dipole-dipole data and smaller uncertainties are chosen for Wenner-Schlumberger data. This is because dipole-dipole surveys have weaker currents.

In [8]:
dc_app.interact_set_uncertainty()

interactive(children=(FloatText(value=5.0, description='percentage'), FloatText(value=0.0, description='floor'…

\begin{align}
\phi_D(m) = |\frac{F(m) - d}{\sigma}|^2
\end{align}

## Step 5: Run inversion

Here, we define parameters necessary to complete the inversion. These parameters are:

- $\rho_0$: initial resistivity model
- $\rho_{ref}$: reference resistivity model
- $\alpha_s$: controls how much the inversion wants to recover a model similar to the reference model
- $\alpha_x$: controls how much the inversion wants to recover a model that is horizontally smooth
- $\alpha_z$: controls how much the inversion wants to recover a model that is vertically smooth
- `maxIter`: maximum number of iteration (10-15 is ideal)
- `chifact`: chifactor for the target misfit
- `beta0_ratio`: ratio to set the initial beta (default value is 10)
- `coolingFactor`: cooling factor to cool beta (default value is 2)
- `n_iter_per_beta`: # of interation for each beta value 
- `run`: run inversion if this is checked

Here are some things to consider when choosing these parameters:

- If you make $\alpha_s$ much larger than $\alpha_x$ and $\alpha_z$, the inversion will try very hard to give you an answer that is similar to the reference model. If you make $\alpha_x$ and $\alpha_z$ much larger than $\alpha_s$, the inversion will try very hard to find a smooth model that explains the data. To start, it is good to set $\alpha_s = 0.01$, $\alpha_x = 1$ and $\alpha_z = 1$.
- If you set *coolingFactor* to a number equal or larger than 4, you should set the *n_iter_per_beta* to a value of 2. This makes sure you solve the problem accurately.
- If the inversion only requires a few iterations, your uncertainties may be too large.
- If your inversion does not reach *target misfit*, your uncertainties may be too small. The number of layers and their thicknessess may also be incorrect.

\begin{align}
\phi(m) = \phi_d(m) + \beta \phi_m(m)\\
\phi_m(m) = \alpha_s\int (m - m_{ref})^2 dV + \alpha_x \int (\frac{\partial m}{\partial x})^2 dV
\end{align}

\begin{align}
\phi_m(m) = \alpha_s\sum m_i^2 dV + \alpha_x \sum (\frac{\partial m}{\partial x}_i)^2 dV
\end{align}

In [9]:
dc_app.interact_run_inversion()

interactive(children=(FloatText(value=15.0, description='$\\rho_0$'), FloatText(value=15.0, description='$\\rh…

## Step 6: Explore inversion results

- `iteration`: inversion iteration
- `curve type`:type of the curve (this is active when `plot type`=`misfit_curve`)
- `scale`: linear or log scale (this is active when `plot type`=`misfit_curve`)
- `plot type`: type of the plot
- $\rho_{min}$, $\rho_{max}$: colorbar limits
- `aspect ratio`: aspect ratio for data misfit plot
- `show grid?`: show mesh grid when this is checked
- `show core?`: show only core domain when this is checked. this is better to interpret results.
- `reverse colormap?`: Reverse the colormap

In [10]:
dc_app.interact_plot_inversion_results()

interactive(children=(IntSlider(value=1, continuous_update=False, description='iteration', max=8, min=1), Togg…

## Step 7: Run inversion to compute DOI index

Depth of investigation (DOI) index can be computed by following equation (Oldenburg and Li, 1999):

$$ \text{doi index} = \frac{m^1-m^2}{m_{ref}^1-m_{ref}^2}$$

where 

- $m^1$: inversion model 1 (from Step5)
- $m^2$: inversion model 2 (same inversion parameters with model 1 except for `m0` and `mref`
- $m_{ref}^1$: reference model 1 (used for Step 5)
- $m_{ref}^2$: reference model 2 (=$m_{ref}^1 \times$ factor)

Here a constant factor is multiplied to generate a new reference model. 
Below app will run inversion to obtain another inversion model ($m^2$), which will allow us to 
compute DOI index in the following app. 

### Parameters
- `factor`: constant factor to compute a new reference model
- `run`: if checked, then a new inverion will run

In [11]:
dc_app.interact_run_doi()

interactive(children=(FloatText(value=0.1, description='factor'), IntSlider(value=5, description='doi_iter', m…

##  Step 8: Plot DOI index


- `plot type`: type of the plot
- `scale`: linear or log scale (this is active when `plot type`=`models` or `final`)
- $\rho_{min}$, $\rho_{max}$: colorbar limits
- `doi_level`: level of the doi index
- `aspect_ratio`: vertical to horizontal ratio
- `show grid?`: show mesh grid when this is checked
- `show core?`: show only core domain when this is checked
- `reverse colormap?`: Reverse the colormap for model plots

In [12]:
dc_app.interact_plot_doi_results()

interactive(children=(ToggleButtons(description='plot type', index=1, options=('models', 'doi', 'final'), valu…